# Ayudantía Web Services & Regex

Realizaremos la actividad correspondiente al semestre 2017-1.

## Parte 1: `Regex`

Mientras los ayudantes de Programación Avanzada creaban la actividad número 15, ~~derramaron su café sobre el computador~~ 
fueron hackeados por GEPANI, quien se encargó de estropear el enunciado de la actividad. Como los ayudantes son un tanto 
perezosos, dejaron el enunciado así y lo subieron al repositorio.

Ahora ustedes, como grandes programadores y ansiosos de terminar la última actividad, tendrán que arreglar
los párrafos corruptos para poder realizarla.
Los párrafos corruptos tienen las siguientes peculiaridades:
    
1. Cada palabra está separada por un @. 

2. En el primer grupo las palabras correctas son las que no tienen
ningún número dentro de ellas.

3. En el segundo, las palabras correctas son las que contienen el string .correcta exactamente, dentro de
ellas.

4. En el tercero las palabras apropiadas son las que tienen un punto y están en minúsculas, como por
ejemplo: ho.la
    
5. Deben seguir el resto de las instrucciones escondidas.

In [ ]:
# importamos el modulo re para trabajar con regex
import re

# abrimos el archivo con el enunciado hackeado
with open("AC15.txt", "r", encoding = "utf-8") as file:
    text = file.read()
    # print(text) # visualizar el texto
    # separamos los parrafos para aplicar los distintos criterios necesarios en cada uno
    parrafos = re.split("\n\n", text)

Luego debemos evaluar que comandos de regex nos son utiles para realizar los pasos que se nos piden. En concreto, deseamos definir una función que nos permita distinguir las palabras correctas para cada criterio.

Se nos dice que el primer grupo de palabras correctas no tienen ningún número dentro de ellas:

In [ ]:
def validar_primero(s):
    pattern = "[0-9]"
    return not bool(re.search(pattern, s))

El segundo grupo, las palabras correctas contienen el string ".correcta" exactamente, dentro de ellas:

In [ ]:
def validar_segundo(s):
    pattern = "[a-zA-Z]*\.correcta[a-zA-Z]*"
    return bool(re.search(pattern, s))

Y en el tercer grupo, las palabras apropiadas son las que tienen un punto y están en minúsculas:

In [ ]:
def validar_tercero(s):
    pattern = "[a-z]*\.[a-z]*"
    return bool(re.match(pattern, s))

Ahora debemos ser capaces de recorrer las palabras separadas por @ y filtrarlas segun un criterio:

In [ ]:
def filtrar_palabras(parrafo, criterio):
    palabras = re.split("@", parrafo)
    for palabra in palabras:
        if criterio(palabra):
            yield palabra

La siguiente función solamente ayuda a que quede más bonito el output

In [ ]:
def nice_format(palabra):
    if palabra[-1] == "\n":
        return palabra[:-1] + " "
    else:
        return palabra + " "

Finalmente debemos aplicar los filtros sobre los distintos parrafos para obtener el enunciado arreglado y así poder escribirlo en un archivo nuevo.

In [ ]:
with open("enunciado.txt", "w", encoding="utf-8") as file:
    enunciado = []
    for palabra in filtrar_palabras(parrafos[0], validar_primero):
        # el primer filtro nos entrega las palabras tal como las queremos
        enunciado.append(nice_format(palabra))
    enunciado.append("\n")
    for palabra in filtrar_palabras(parrafos[1], validar_segundo):
        # el segundo filtro requiere que removamos la ocurrencia de '.correcta'
        palabra = re.sub("\.correcta", "", palabra)
        enunciado.append(nice_format(palabra))
    enunciado.append("\n")
    for palabra in filtrar_palabras(parrafos[2], validar_tercero):
        # el tercer filtro requiere que removamos la ocurrencia de '\.'
        palabra = re.sub("\.","", palabra + " ")
        enunciado.append(nice_format(palabra))
    enunciado.append("\n")
    # finalmente escribimos todo en el archivo, juntando los parrafos
    file.write("".join(enunciado))

## Parte 2: `Web Service`

Para hacer hacer las búsquedas y obtener el contenidos de las páginas:
1. El parámetro ’action’ debe tener el valor ’query’ (https://en.wikipedia.org/w/api.php?action=help&modules=query).
2. El parámetro ’prop’ debe tener las propiedades que quieres extraer
3. El parámetro ’export’ debe ser ’true’
4. Los requests deben ser realizados a https://es.wikipedia.org/w/api.php?
5. Cada método GET se hace a esta misma url, con los parámetros encapsulados en un diccionario. Por ejemplo, GET (.../w/api.php?, params=’action’: ’query’,’titles’: ’chile’,’prop’:’extracts’, ’format’:’json’)
6. El siguiente link muestra en el navegador como se vería una request de ejemplo. https://es.wikipedia.org/w/api.php?action=query&titles=chile&prop=extracts&export=true


In [ ]:
# importamos 
import json
import requests
import os
import time


class PrograPedia:

    def __init__(self):
        self.dic = {}
        self.url = "https://es.wikipedia.org/w/api.php?"
        self.path = "PrograPaginas.pp"

    def search_wiki(self, word):
        value = requests.get(self.url, params={'action': 'query',
                                               'titles': word,
                                               'prop': 'extracts',
                                               'export': True,
                                               'explaintext': '',
                                               'format': 'json'})

        key = str(list(value.json()['query']['pages'].keys())[0])
        target_dic = value.json()['query']['pages'][key]
        print(target_dic)
        my_dic = {word: {'title': word,
                         'pageid': target_dic['pageid'],
                         'url': None,
                         'content': target_dic['extract']}
                      }
        print(my_dic)
        self.dic.update(my_dic)
        return my_dic[word]['content']

    def search(self, word):
        if word in self.dic:
            return self.dic[word]["content"]
        else:
            return self.search_wiki(word)

    def save_dict(self):
        with open(self.path, "w") as file:
            file.write(json.dumps(self.dic))

    def menu(self):
        print("Bienvenido a PrograPedia!\n\n")
        fin = False
        while(not fin):
            accion = input("Que deseas hacer?\n1) Buscar\n2) Salir\n> ")
            if accion == "1":
                busqueda = input("Que deseas buscar?: ")
                t = time.time()
                result = self.search(busqueda)
                print(result)
                print(time.time() - t)
            elif accion == "2":
                print("Adios!")
                self.save_dict()
                fin = True
            else:
                print("Error: Accion inexistente..")



if __name__ == '__main__':
    prograpedia = PrograPedia()
    prograpedia.menu()